In [ ]:
## Building Model

In [ ]:
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical computations
import matplotlib.pyplot as plt  # For creating static, animated, and interactive visualizations
import seaborn as sns  # For statistical data visualization based on Matplotlib
from tabulate import tabulate  # For pretty-printing tabular data

from sklearn import metrics  # For model evaluation metrics
from sklearn.ensemble import RandomForestClassifier  # For building a random forest classifier
from sklearn.model_selection import train_test_split  # To split data into train and test sets
from sklearn.metrics import recall_score  # To calculate recall score for classification models
from sklearn.metrics import classification_report  # To generate classification performance report
from sklearn.metrics import confusion_matrix  # To create a confusion matrix for model evaluation
from sklearn.tree import DecisionTreeClassifier  # For building a Decision Tree classifier
from imblearn.combine import SMOTEENN  # For handling class imbalance using SMOTE and ENN
import pickle # For storing and loading Python objects

In [ ]:
# Load dataset from CSV file
df = pd.read_csv(r"D:\Stuff\Data Science\Machine Learning Models\Customer_churn_model\WA_Fn-UseC_-Telco-Customer-Churn.csv")
# Display the first few rows of the dataframe
df.head()

##### As it is obsererved in the EDA part 
- It is observed that `TotalCharges` is of type **object**, whereas it should be of type **float**.
- The `SeniorCitizen` column should ideally be of type **object** instead of **int64**, since it contains values `0` and `1`, which represent categories rather than numerical values. Changing the data type helps reduce confusion during calculations.
- All other column data types appear to be appropriate.

In [ ]:
# Convert 'TotalCharges' column to numeric, coercing errors to NaN
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')
# Convert 'SSeniorCitizen' column to categorical type
df.SeniorCitizen = df.SeniorCitizen.astype('category')
# Display the data types of each column again to confirm changes 
df.dtypes

#### It is observed that the 'TotalCharges' column has some NaN values.( In EDA File )

In [ ]:
# Drop rows with null values in any column
df.dropna(how = 'any', inplace = True)

In [ ]:
# Create a new DataFrame to avoid modifying the original
new_df = df.copy()

In [ ]:
# Define tenure group labels in 12-month intervals (e.g., '1 - 12', '13 - 24', ...)
labels = ["{0} - {1}".format(i, i + 11) for i in range(1, 72, 12)]

# Categorize 'tenure' into discrete bins using the defined intervals
# right=False → interval includes the left value and excludes the right (e.g., [1,13) )
new_df['tenure_group'] = pd.cut(new_df.tenure, bins=range(1, 80, 12), right=False, labels=labels)

# Display the distribution of customers across tenure groups
new_df['tenure_group'].value_counts()

In [ ]:
# Remove the 'tenure' and 'customerID' columns from the DataFrame 
new_df.drop(columns = ['customerID','tenure'], axis=1, inplace=True)
new_df.head()

In [ ]:
# Apply label encoding to churn column
new_df['Churn'] = np.where(new_df['Churn']=='Yes',1,0)
new_df.head()

In [ ]:
# Aplly label encoding to categorical columns
new_df_dummies = pd.get_dummies(new_df)
new_df_dummies = new_df_dummies.astype(int)
new_df_dummies.head()

In [ ]:
# Dropping the 'Churn' column to use all other columns as features
x = new_df_dummies.drop(columns = 'Churn',axis = 1)

# Extracting the 'Churn' column as the target variable
y = new_df_dummies['Churn']

In [ ]:
# Display the shape of the feature set and target variable
print(x.shape)
print(y.shape)

##### Split test and train data

In [ ]:
# Splitting the dataset into training and testing sets with a 70-30 split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

### Decision Tree Classifier

In [ ]:
# Initializing a DecisionTreeClassifier model
model_dt = DecisionTreeClassifier(criterion="gini", random_state=42, max_depth=6, min_samples_leaf=8)

# - criterion="gini": The Gini impurity measure is used to split nodes.
# - random_state=42: Ensures reproducibility of results.
# - max_depth=6: Limits the depth of the tree to prevent overfitting.
# - min_samples_leaf=8: Ensures each leaf node has at least 8 samples to avoid overfitting.

In [ ]:
# Fitting the DecisionTreeClassifier model to the training data
model_dt.fit(x_train, y_train)

In [ ]:
# Making predictions using the trained DecisionTreeClassifier model on the test data (x_test)
y_pred = model_dt.predict(x_test)
print(f"Total Rows: {y_pred.shape[0]}")
print(y_pred)

In [ ]:
# Evaluate model accuracy on the test set
print(f"Accuracy: {(model_dt.score(x_test, y_test))*100 :.2f} %")

In [ ]:
def pretty_classification_report(y_true, y_pred, labels=None):
    report = classification_report(y_true, y_pred, labels=labels)
    print("-" * 60)
    print(report)
    print("-" * 60)

# Display precision, recall, f1-score, and support for each class (0: No Churn, 1: Churn)
print(classification_report(y_test, y_pred, labels=[0,1]))

### Model Evaluation Insight

As you can see, the **accuracy** is relatively low. Since this is an **imbalanced dataset**, we shouldn't rely on accuracy as a performance metric. Accuracy can be misleading in such cases — it's often considered **"cursed"** when class distributions are skewed.

#### What should we focus on instead?
We need to evaluate the model using:
- **Precision**
- **Recall**
- **F1-score**

Especially for the **minority class (Class 1: Churned customers)**.

#### Observation:
- Precision, recall, and F1-score for churned customers are significantly low.
- This indicates the model is not effectively capturing the patterns related to customer churn.

---

### Next Step: Apply SMOTEENN
We'll now move ahead to apply **SMOTEENN**, a technique that combines:
- **SMOTE (Synthetic Minority Oversampling Technique)** to balance the minority class, and
- **ENN (Edited Nearest Neighbors)** to clean noisy samples

This should help improve model performance on the minority class.


In [ ]:
# Apply SMOTEENN to balance the dataset by oversampling the minority class and cleaning noisy samples
sm = SMOTEENN()
x_smot, y_smot = sm.fit_resample(x, y)

In [ ]:
# Split the resampled data into training and testing sets (80/20 split)
xr_train, xr_test, yr_train, yr_test = train_test_split(x_smot, y_smot, test_size=0.2)

In [ ]:
# Initialize Decision Tree with specified hyperparameters on SMOTEENN data
model_dt_smote = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=6, min_samples_leaf=8)

In [ ]:
# Train the Decision Tree model on the resampled training data
model_dt_smote.fit(xr_train, yr_train)

# Predict on the test set
yr_predict = model_dt_smote.predict(xr_test)

# Get accuracy score on resampled test data
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(f"Model Score: {model_score_r:.2%}")

# Display classification report for precision, recall, f1-score
print(metrics.classification_report(yr_test, yr_predict))

In [ ]:
# Get the confusion matrix
cm = confusion_matrix(yr_test, yr_predict)

# Plot it using seaborn
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Display the confusion matrix for the resampled test set
print("confusion_matrix: ","\n",metrics.confusion_matrix(yr_test, yr_predict))

### Random Forest Classifier

In [ ]:
# Initialize a Random Forest classifier with 100 trees, Gini index for splitting, and specific hyperparameters
model_rf = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=100, max_depth=6, min_samples_leaf=8)

In [ ]:
# Fit the Random Forest model to the training data
model_rf.fit(x_train, y_train)

In [ ]:
# Predict the target labels using the trained Random Forest model on the test data
y_pred = model_rf.predict(x_test)

In [ ]:
# Evaluate the accuracy of the Random Forest model on the test data
print(f"Random forest Model Score: {model_rf.score(x_test, y_test):.2%}")

In [ ]:
# Generate and display the classification report for the Random Forest model's predictions
print(classification_report(y_test, y_pred, labels=[0, 1]))

In [ ]:
# Apply SMOTEENN (Synthetic Minority Over-sampling Technique + Edited Nearest Neighbors) to balance the dataset by generating synthetic samples and removing noise
x_smot1, y_smot1 = sm.fit_resample(x, y)

In [ ]:
# Split the resampled dataset with 80% for training and 20% for testing
xrf_train, xrf_test, yrf_train, yrf_test = train_test_split(x_smot1, y_smot1, test_size=0.2)

In [ ]:
# Initialize a RandomForestClassifier model with specified parameters
model_rf_smote = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=100, max_depth=6, min_samples_leaf=8)

# - 100 trees (n_estimators=100)
# - Gini impurity for splitting (criterion='gini')
# - Random state set to 100 for reproducibility
# - Maximum depth of the trees set to 6 (max_depth=6)
# - Minimum samples per leaf set to 8 (min_samples_leaf=8)

In [ ]:
# Fit the Random Forest model on the resampled training data
model_rf_smote.fit(xrf_train, yrf_train)

In [ ]:
# Predict the target variable using the trained Random Forest model
yr_predict1 = model_rf_smote.predict(xrf_test)

In [ ]:
# Evaluate the model's performance by calculating the accuracy on the resampled test set
model_score_r1 = model_rf_smote.score(xrf_test, yrf_test)
print(f"Random Forest Model Score: {model_score_r1:.2%}") # Print the model's accuracy score on the resampled test set

In [ ]:
# Print the classification report to evaluate precision, recall, f1-score, etc., for each class
print(metrics.classification_report(yrf_test, yr_predict1))

In [ ]:
# Print the confusion matrix to evaluate the performance of the model on the resampled test set
print(metrics.confusion_matrix(yrf_test, yr_predict1))

In [ ]:
# Get the confusion matrix
cm1 = confusion_matrix(yrf_test, yr_predict1)

# Plot it using seaborn
plt.figure(figsize=(5,4))
sns.heatmap(cm1, annot=True, fmt='d', cmap='Blues', xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Display the confusion matrix for the resampled test set
print("confusion_matrix: ","\n",metrics.confusion_matrix(yrf_test, yr_predict1))

### Save the model

In [ ]:
# # Save the trained Random Forest model with SMOTEENN applied to a file using pickle
# filename = 'churn.model'
# # filename = 'model.sav' # Define the filename for saving the model
# pickle.dump(model_rf_smote, open(filename, 'wb'))

##### Open loaded Model to check 

In [ ]:
# # Load the previously saved Random Forest model from the file
# load_model = pickle.load(open(filename, 'rb'))

# # Evaluate the loaded model's performance on the test data
# print(f"Model Score: {load_model.score(xrf_test, yrf_test):.2%}")

**Our final model, i.e., the RF Classifier with SMOTEENN, is now ready and saved in the file `churn.model`. We will use this model to prepare APIs, enabling us to access it from the UI.**

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split

# # Separate features and target
# X = new_df.drop('Churn', axis=1)
# y = new_df['Churn']  # Already encoded as 1/0

# # Identify categorical and numeric columns
# cat_cols = X.select_dtypes(include='object').columns.tolist()
# num_cols = X.select_dtypes(exclude='object').columns.tolist()

# # Build preprocessing pipeline
# preprocessor = ColumnTransformer(transformers=[
#     ('cat', OneHotEncoder(drop='first'), cat_cols),  # drop='first' to avoid dummy trap
#     ('num', 'passthrough', num_cols)
# ])

# # Combine preprocessing + model in a full pipeline
# clf_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', RandomForestClassifier(random_state=42))
# ])

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Fit the pipeline
# clf_pipeline.fit(X_train, y_train)

# # Predict on new data
# y_pred = clf_pipeline.predict(X_test)

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# import joblib

# # Assuming you already trained your model/pipeline
# pipe = Pipeline([
#     ('clf', RandomForestClassifier())
# ])

# pipe.fit(xr_train1, yr_train1)

# # Save it
# joblib.dump(pipe, 'model.sav')
# print("✅ Model saved successfully!")\

In [ ]:
# import joblib
# model = joblib.load('model.sav')
# print(type(model))  # It should be <class 'sklearn.ensemble.RandomForestClassifier'>